# WiDS Datathon 2022

## Overview

The WiDS Datathon 2022 focuses on a prediction task involving roughly 100k observations of **building energy usage records** collected over 7 years and a number of states within the United States. The dataset consists of building characteristics (e.g. floor area, facility type etc), weather data for the location of the building (e.g. annual average temperature, annual total precipitation etc) as well as the energy usage for the building and the given year, measured as **Site Energy Usage Intensity (Site EUI)**.

**Each row in the data corresponds to the a single building observed in a given year**. The task is to **predict the Site EUI for each row**, given the characteristics of the building and the weather data for the location of the building.

## Data Dictionary

  - `id`: building id
  - `Year_Factor`: anonymized year in which the weather and energy usage factors were observed
  - `State_Factor`: anonymized state in which the building is located
  - `building_class`: building classification
  - `facility_type`: building usage type
  - `floor_area`: floor area (in square feet) of the building
  - `year_built`: year in which the building was constructed
  - `energy_star_rating`: the energy star rating of the building
  - `ELEVATION`: elevation of the building location
  - `january_min_temp`: minimum temperature in January (in Fahrenheit) at the location of the building
  - `january_avg_temp`: average temperature in January (in Fahrenheit) at the location of the building
  - `january_max_temp`: maximum temperature in January (in Fahrenheit) at the location of the building
`cooling_degree_days`: cooling degree day for a given day is the number of degrees where the daily average temperature exceeds 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
`heating_degree_days`: heating degree day for a given day is the number of degrees where the daily average temperature falls under 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
  - `precipitation_inches`: annual precipitation in inches at the location of the building
  - `snowfall_inches`: annual snowfall in inches at the location of the building
  - `snowdepth_inches`: annual snow depth in inches at the location of the building
  - `avg_temp`: average temperature over a year at the location of the building
  - `days_below_30F`: total number of days below 30 degrees Fahrenheit at the location of the building
  - `days_below_20F`: total number of days below 20 degrees Fahrenheit at the location of the building
  - `days_below_10F`: total number of days below 10 degrees Fahrenheit at the location of the building
  - `days_below_0F`: total number of days below 0 degrees Fahrenheit at the location of the building
  - `days_above_80F`: total number of days above 80 degrees Fahrenheit at the location of the building
  - `days_above_90F`: total number of days above 90 degrees Fahrenheit at the location of the building
  - `days_above_100F`: total number of days above 100 degrees Fahrenheit at the location of the building
  - `days_above_110F`: total number of days above 110 degrees Fahrenheit at the location of the building
  - `direction_max_wind_speed`: wind direction for maximum wind speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
  - `direction_peak_wind_speed`: wind direction for peak wind gust speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
  - `max_wind_speed`: maximum wind speed at the location of the building
  - `days_with_fog`: number of days with fog at the location of the building

## Environment Setup

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max.columns', None)
pd.set_option('display.max.rows', 50)
pd.set_option('display.precision', 5)

In [3]:
if 'notebooks' in os.getcwd():
    os.chdir('..')

DATA_PATH = os.getcwd() + '/data/'
fname = 'train.csv'

## Data Exploration

### Reading in Data

In [6]:
df = pd.read_csv(DATA_PATH + fname, header='infer', low_memory=False)
df.columns = [x.lower() for x in df.columns]
print(df.shape)
df.head()

(75757, 64)


,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,68,35,50.58929,73,40,53.69355,80,41,55.5,78,46,56.85484,84,50,60.5,90,52,62.72581,84,52,62.16129,85,52,64.65,90,47,63.01613,83,43,53.8,72,36,49.27419,71,115,2960,16.59,0.0,0,56.9726,0,0,0,0,14,0,0,0,1.0,1.0,1.0,NaN,248.68262,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,68,35,50.58929,73,40,53.69355,80,41,55.5,78,46,56.85484,84,50,60.5,90,52,62.72581,84,52,62.16129,85,52,64.65,90,47,63.01613,83,43,53.8,72,36,49.27419,71,115,2960,16.59,0.0,0,56.9726,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,26.50015,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,68,35,50.58929,73,40,53.69355,80,41,55.5,78,46,56.85484,84,50,60.5,90,52,62.72581,84,52,62.16129,85,52,64.65,90,47,63.01613,83,43,53.8,72,36,49.27419,71,115,2960,16.59,0.0,0,56.9726,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,24.69362,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,68,35,50.58929,73,40,53.69355,80,41,55.5,78,46,56.85484,84,50,60.5,90,52,62.72581,84,52,62.16129,85,52,64.65,90,47,63.01613,83,43,53.8,72,36,49.27419,71,115,2960,16.59,0.0,0,56.9726,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,48.40693,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,68,35,50.58929,73,40,53.69355,80,41,55.5,78,46,56.85484,84,50,60.5,90,52,62.72581,84,52,62.16129,85,52,64.65,90,47,63.01613,83,43,53.8,72,36,49.27419,71,115,2960,16.59,0.0,0,56.9726,0,0,0,0,14,0,0,0,1.0,1.0,1.0,NaN,3.89940,4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 64 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year_factor                75757 non-null  int64  
 1   state_factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   elevation                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

NoneType

Columns with nulls - 

  - `year_built`
  - `energy_star_rating`
  - `direction_max_wind_speed`
  - `direction_peak_wind_speed`
  - `max_wind_speed`
  - `days_with_fog`

### Categorical Attributes

  - `year_factor`
  - `state_factor`
  - `building_class`
  - `facility_type`

In [13]:
cat_cols = ['year_factor', 'state_factor', 'building_class', 'facility_type']
for col in cat_cols:
    print(col, df[col].nunique())
    print(df[col].value_counts())
    print(2*len(col)*'-')

year_factor 6
6    22449
5    18308
4    12946
3    10879
2     9058
1     2117
Name: year_factor, dtype: int64
----------------------
state_factor 7
State_6     50840
State_11     6412
State_1      5618
State_2      4871
State_4      4300
State_8      3701
State_10       15
Name: state_factor, dtype: int64
------------------------
building_class 2
Residential    43558
Commercial     32199
Name: building_class, dtype: int64
----------------------------
facility_type 60
Multifamily_Uncategorized              39455
Office_Uncategorized                   12512
Education_Other_classroom               3860
Lodging_Hotel                           2098
2to4_Unit_Building                      1893
                                       ...  
Food_Service_Other                        17
Mixed_Use_Predominantly_Residential        9
Public_Assembly_Stadium                    9
Service_Drycleaning_or_Laundry             9
Lodging_Uncategorized                      5
Name: facility_type, Length: 60